# MongoDB Handling

After installing the MongoDB server in your machine, you can use this notebook for handling the initial processes with the database.

Specifically, in this step, we utilize Python's `pymongo` library to exploit its capabilities for MongoDB server interaction.

**Important Note: Be sure that the MongoDB server is up and running as a service in the background.**

For example, in macOS, to run MongoDB (i.e. the mongod process) as a service, run:

* `brew services start mongodb-community`

To stop a mongod running as a macOS service, use the following command as needed:

* `brew services stop mongodb-community`

To install MongoDB in your system, follow the instructions here:

* https://www.mongodb.com/docs/manual/administration/install-community/


**Note:** You can modify any of the processes below, however, you have to explain your thoughts.

In [27]:
# import library for various processes with the OS
import os

## Load configuration

In [28]:
# import library for yaml handling
import yaml

In [29]:
config_path = os.path.join(os.getcwd(), "config.yml")

with open(config_path) as file:
    config = yaml.load(file, Loader=yaml.FullLoader)

## MongoDB database instantiation

The relevant information for the MongoDB client connection, the database name, and collection name is located in the configuration file.

```
# DB Connection with the uri (host)
client: "mongodb://localhost:27017/"

# db name
db: "aiot_course"

# db collection
col: "NAME YOUR COLLECTION"
```

In [30]:
# import library for hanlding the MongoDB client
import pymongo
# import library for retrieving datetime
from datetime import datetime

### Create the database

To create a database in MongoDB, start by creating a MongoClient object, then specify a connection URL with the correct ip address and the name of the database you want to create.

MongoDB will create the database if it does not exist, and make a connection to it.

In [31]:
config = {
    "client": "mongodb://localhost:27017/",
    "db": "AIoT_project",
    "col": "AIoT_project"
}

client = pymongo.MongoClient(config["client"])

In [32]:
db = client[config["db"]]

### Instantiate the collection

To create a collection in MongoDB, use database object and specify the name of the collection you want to create.

MongoDB will create the collection if it does not exist.

In [33]:
col = db[config["col"]]

Initially, no collection will be shown in MongoDB before you enter the first document!

## Create the data collection

Uploading the gathered data to MongoDB collection. The data directory structure should be as follows:

```
.
└── data/
    ├── class_A/
    │   ├── data_A_01.csv
    │   ├── data_A_02.csv
    │   └── ..
    ├── class_B/
    │   ├── data_B_01.csv
    │   ├── data_B_02.csv
    │   └── .
    └── class ...
```

In [34]:
# import library for hanlding the csv data and transformations
import pandas as pd
import json

Get data path:

In [35]:
data_path = os.path.join(os.getcwd(), "windowed_data")
print(data_path)

g:\Other computers\My Computer\8 Εξάμηνο\Αλγοριθμικές Θεμελιώσεις Δικτύων Αισθητήρων\Human Gesture Recognition Project\windowed_folder


List all files in a path:

In [36]:
classes_folders_list = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]
print(classes_folders_list)

['class_A', 'class_B', 'class_D', 'class_C']


In [37]:
# print files in folder
folder_path = os.path.join(data_path, classes_folders_list[0])
files_in_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(files_in_folder)

['data_A_1_window1.csv', 'data_A_1_window2.csv', 'data_A_1_window3.csv', 'data_A_1_window4.csv', 'data_A_1_window5.csv', 'data_A_1_window6.csv', 'data_A_1_window7.csv', 'data_A_2_window1.csv', 'data_A_2_window2.csv', 'data_A_2_window3.csv', 'data_A_2_window4.csv', 'data_A_2_window5.csv', 'data_A_2_window6.csv', 'data_A_2_window7.csv', 'data_A_3_window1.csv', 'data_A_3_window2.csv', 'data_A_3_window3.csv', 'data_A_3_window4.csv', 'data_A_3_window5.csv', 'data_A_3_window6.csv', 'data_A_3_window7.csv', 'data_A_4_window1.csv', 'data_A_4_window2.csv', 'data_A_4_window3.csv', 'data_A_4_window4.csv', 'data_A_4_window5.csv', 'data_A_4_window6.csv', 'data_A_4_window7.csv', 'data_A_5_window1.csv', 'data_A_5_window2.csv', 'data_A_5_window3.csv', 'data_A_5_window4.csv', 'data_A_5_window5.csv', 'data_A_5_window6.csv', 'data_A_5_window7.csv', 'data_A_5_window8.csv', 'data_A_6_window1.csv', 'data_A_6_window2.csv', 'data_A_6_window3.csv', 'data_A_6_window4.csv', 'data_A_6_window5.csv', 'data_A_6_windo

Each document in the MongoDB database should have the following schema:

```json
{
  "data": {
    "acc_x": ["array", "of", "values"],
    "acc_y": ["array", "of", "values"],
    "acc_z": ["array", "of", "values"],
  },
  "label": "The label of the instance",
  "datetime": "MongoDB datetime object (it can be generated with the datetime.datetime.now() function"
}
```

Accordingly, if you are using gyroscope or both accelerometer and gyroscope, the following order and naming of the sensor keys should be defined:

* for gyroscope: `gyr_x`, `gyr_y`, `gyr_z` for the three axes
* for accelerometer and gyroscope: `acc_x`, `acc_y`, `acc_z`, `gyr_x`, `gyr_y`, `gyr_z` for the six axes

**Note: Be careful, the document is mandatory to have the aforementioned schema, in order to argue and proceed with the rest of the processes later on, in data engineering, plotting, etc.**

In [38]:
from utils import df_rebase

## Provide the code to upload the data to MongoDB

In [39]:
def upload_data_to_mongodb(data_path, collection):
    classes_folders_list = [f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))]

    for class_folder in classes_folders_list:
        folder_path = os.path.join(data_path, class_folder)
        files_in_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

        for file_name in files_in_folder:
            file_path = os.path.join(folder_path, file_name)
            df = pd.read_csv(file_path)

            # Extract data from DataFrame
            data = {
                "data": {
                    "x-axis (g)": df['x-axis (g)'].tolist(),
                    "y-axis (g)": df['y-axis (g)'].tolist(),
                    "z-axis (g)": df['z-axis (g)'].tolist(),
                    "x-axis (deg/s)": df['x-axis (deg/s)'].tolist(),
                    "y-axis (deg/s)": df['y-axis (deg/s)'].tolist(),
                    "z-axis (deg/s)": df['z-axis (deg/s)'].tolist(),
                    
                },
                "label": class_folder,
                "datetime": datetime.now()
            }

            collection.insert_one(data)
            print(f"Uploaded {file_name} from {class_folder}")

# Run the function to upload data
upload_data_to_mongodb(data_path, col)

Uploaded data_A_1_window1.csv from class_A
Uploaded data_A_1_window2.csv from class_A
Uploaded data_A_1_window3.csv from class_A
Uploaded data_A_1_window4.csv from class_A
Uploaded data_A_1_window5.csv from class_A
Uploaded data_A_1_window6.csv from class_A
Uploaded data_A_1_window7.csv from class_A
Uploaded data_A_2_window1.csv from class_A
Uploaded data_A_2_window2.csv from class_A
Uploaded data_A_2_window3.csv from class_A
Uploaded data_A_2_window4.csv from class_A
Uploaded data_A_2_window5.csv from class_A
Uploaded data_A_2_window6.csv from class_A
Uploaded data_A_2_window7.csv from class_A
Uploaded data_A_3_window1.csv from class_A
Uploaded data_A_3_window2.csv from class_A
Uploaded data_A_3_window3.csv from class_A
Uploaded data_A_3_window4.csv from class_A
Uploaded data_A_3_window5.csv from class_A
Uploaded data_A_3_window6.csv from class_A
Uploaded data_A_3_window7.csv from class_A
Uploaded data_A_4_window1.csv from class_A
Uploaded data_A_4_window2.csv from class_A
Uploaded da